In [ ]:
from fusiontimeseries.finetuning.preprocessing.utils import (
    create_windowed_train_and_test_flux_ts_dataframes,
)

train_flux_df, val_flux_df = create_windowed_train_and_test_flux_ts_dataframes(
    n_discretation_quantiles=5, test_set_size=0.1
)
print(f"Training set size: {len(train_flux_df)}")
print(f"Validation set size: {len(val_flux_df)}")

In [ ]:
chronos2_hyperparameters = {
    # https://auto.gluon.ai/stable/tutorials/timeseries/forecasting-model-zoo.html#pretrained-models
    "Chronos2": {
        "fine_tune": True,
        "fine_tune_mode": "lora",
        "fine_tune_lr": 0.0005,
        "fine_tune_steps": 7000,
        "fine_tune_batch_size": 64,
        "cross_learning": True,
        "fine_tune_lora_config": {"r": 16, "lora_alpha": 32, "lora_dropout": 0.1},
    }
}

In [ ]:
from autogluon.timeseries import TimeSeriesDataFrame

training_data = TimeSeriesDataFrame.from_data_frame(
    train_flux_df,
    id_column="item_id",
    timestamp_column="timestamp",
)
validation_data = TimeSeriesDataFrame.from_data_frame(
    val_flux_df,
    id_column="item_id",
    timestamp_column="timestamp",
)
print(f"Training set size: {training_data.size}")
print(f"Validation set size: {validation_data.size}")

In [ ]:
from autogluon.timeseries import TimeSeriesPredictor

predictor = TimeSeriesPredictor(
    prediction_length=80, target="target", eval_metric="RMSE", verbosity=4
).fit(
    train_data=training_data,
    tuning_data=validation_data,  # add custom validation data
    hyperparameters={**chronos2_hyperparameters},
    time_limit=3600,
    enable_ensemble=False,
    verbosity=4,
)

In [ ]:
from autogluon.timeseries import TimeSeriesPredictor

raw_predictor_path = input("Enter path to save the predictor: ")
predictor = TimeSeriesPredictor.load(raw_predictor_path)

In [ ]:
# Complete evaluation with automatic saving of all results
from fusiontimeseries.finetuning.evaluation_utils import (
    FinetuningConfig,
    run_complete_evaluation,
)

# Create configuration object
finetuning_config = FinetuningConfig(
    model_name="Chronos2",  # TODO add manually
    prediction_length=predictor.prediction_length,
    target="target",
    eval_metric="RMSE",
    hyperparameters=chronos2_hyperparameters,  # TODO add manually
    time_limit=3600,
    start_context_length=80,
    relevant_tail_length=80,
)

# Run complete evaluation (forecasts, evaluates, plots, and saves everything)
results, json_path, plots_dir = run_complete_evaluation(
    predictor=predictor,
    config=finetuning_config,
    training_data_size=training_data.num_items,
    id="Chronos2_Augmented_Windowed_Train_Val_Finetuning_7000",
    predictor_path=predictor.path if hasattr(predictor, "path") else None,
)

print("\n✅ All results saved!")
print(f"📊 Plots directory: {plots_dir}")
print(f"📄 JSON results: {json_path}")

next experiments include setting prediction length to 80 to have validation window on tuning set for validation error to be an exact RMSE measure for the evaluation constraint.